In [15]:
# Setup
import pandas as pd

raw_data = pd.read_csv('../src/raw/raw_data.csv')

In [20]:
raw_data.head()

,Unnamed: 0,ID,Street,HouseNumber,Box,Floor,City,PostalCode,Region,District,...,Garden,GardenArea,Facades,SwimmingPool,Condition,EPCScore,Latitude,Longitude,PropertyUrl,Property url
0,0,11132404,Rue de Sart,76,NaN,NaN,COURT-SAINT-ETIENNE,1490,WALLONIE,Nivelles,...,1.0,1333.0,4.0,NaN,JUST_RENOVATED,D,50.624459,4.562738,NaN,https://www.immoweb.be/en/classified/house/for...
1,1,10820901,Rue de Trisogne,15,D,NaN,Ciney,5590,WALLONIE,Dinant,...,NaN,736.0,3.0,NaN,AS_NEW,A,50.281860,5.167839,NaN,https://www.immoweb.be/en/classified/villa/for...
2,2,11080049,Chauss�e de Mons,747,NaN,NaN,Anderlecht,1070,BRUSSELS,Brussels,...,NaN,91.0,2.0,0.0,TO_BE_DONE_UP,F,50.829926,4.307101,NaN,https://www.immoweb.be/en/classified/house/for...
3,3,11137923,Braambezielaan,10,NaN,NaN,Gent Wondelgem,9032,FLANDERS,Gent,...,1.0,261.0,2.0,0.0,AS_NEW,D,51.080022,3.702782,NaN,https://www.immoweb.be/en/classified/house/for...
4,4,11139236,Lindanusstraat,39,NaN,NaN,Dendermonde,9200,FLANDERS,Dendermonde,...,1.0,215.0,2.0,NaN,TO_BE_DONE_UP,E,51.028484,4.103270,NaN,https://www.immoweb.be/en/classified/mansion/f...


In [ ]:
raw_data.notnull().sum()

In [ ]:
raw_data.isna().sum() 

In [19]:
raw_data[raw_data['City'].str.contains('�', na=False)]

,Unnamed: 0,ID,Street,HouseNumber,Box,Floor,City,PostalCode,Region,District,...,Garden,GardenArea,Facades,SwimmingPool,Condition,EPCScore,Latitude,Longitude,PropertyUrl,Property url
11,11,11150855,Chauss�e de Ramioul,223,NaN,NaN,FL�MALLE,4400,WALLONIE,Li�ge,...,NaN,140.0,3.0,NaN,GOOD,F,50.583011,5.424205,NaN,https://www.immoweb.be/en/classified/apartment...
12,12,11150854,Chauss�e de Ramioul,223,NaN,NaN,FL�MALLE,4400,WALLONIE,Li�ge,...,NaN,140.0,3.0,NaN,GOOD,F,50.583011,5.424205,NaN,https://www.immoweb.be/en/classified/house/for...
18,18,11128858,Rue du Moulin,257,NaN,NaN,Li�ge,4020,WALLONIE,Li�ge,...,NaN,81.0,2.0,0.0,TO_BE_DONE_UP,F,50.643276,5.604360,NaN,https://www.immoweb.be/en/classified/house/for...
29,29,11109457,Rue des Cytises,46,NaN,NaN,Fl�malle,4400,WALLONIE,Li�ge,...,1.0,509.0,4.0,0.0,TO_RENOVATE,G,50.617085,5.445439,NaN,https://www.immoweb.be/en/classified/house/for...
38,38,11094771,Rue des Bons Buveurs,33,NaN,NaN,MONTEGN�E,4420,WALLONIE,Li�ge,...,1.0,250.0,2.0,0.0,TO_RENOVATE,D,50.639158,5.535229,NaN,https://www.immoweb.be/en/classified/house/for...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18260,18260,11046538,Rue Emile Vandervelde,53,NaN,0.0,Fl�ron Roms�e,4624,WALLONIE,Li�ge,...,NaN,NaN,3.0,0.0,JUST_RENOVATED,B,50.611690,5.669116,NaN,https://www.immoweb.be/en/classified/apartment...
18261,18261,11046537,Rue Emile Vandervelde,53,NaN,0.0,Fl�ron Roms�e,4624,WALLONIE,Li�ge,...,NaN,NaN,3.0,0.0,JUST_RENOVATED,B,50.611690,5.669116,NaN,https://www.immoweb.be/en/classified/apartment...
18262,18262,11046536,Rue Emile Vandervelde,53,NaN,0.0,Fl�ron Roms�e,4624,WALLONIE,Li�ge,...,NaN,NaN,3.0,0.0,JUST_RENOVATED,B,50.611690,5.669116,NaN,https://www.immoweb.be/en/classified/apartment...
18273,18273,11004054,Rue Mathieu de Lexhy,300,A,1.0,Gr�ce-hollogne,4460,WALLONIE,Li�ge,...,NaN,NaN,3.0,NaN,GOOD,D,50.647979,5.494844,NaN,https://www.immoweb.be/en/classified/loft/for-...


In [24]:
raw_data.columns

Index(['ID', 'Street', 'HouseNumber', 'Box', 'Floor', 'City', 'PostalCode',
       'Region', 'District', 'Province', 'PropertyType', 'PropertySubType',
       'Price', 'SaleType', 'ConstructionYear', 'BedroomCount', 'LivingArea',
       'KitchenType', 'Furnished', 'Fireplace', 'Terrace', 'TerraceArea',
       'Garden', 'GardenArea', 'Facades', 'SwimmingPool', 'Condition',
       'EPCScore', 'Latitude', 'Longitude', 'PropertyUrl', 'Property url'],
      dtype='object')

In [8]:
raw_data['PropertySubType'].unique()


array(['HOUSE', 'VILLA', 'MANSION', 'APARTMENT_BLOCK',
       'MIXED_USE_BUILDING', 'EXCEPTIONAL_PROPERTY', 'TOWN_HOUSE',
       'MANOR_HOUSE', 'BUNGALOW', 'COUNTRY_COTTAGE', 'FARMHOUSE',
       'CASTLE', 'OTHER_PROPERTY', 'CHALET', 'APARTMENT', 'PENTHOUSE',
       'DUPLEX', 'FLAT_STUDIO', 'SERVICE_FLAT', 'TRIPLEX', 'KOT',
       'GROUND_FLOOR', 'LOFT'], dtype=object)

In [16]:
raw_data['District'].unique()

array(['Nivelles', 'Dinant', 'Brussels', 'Gent', 'Dendermonde',
       'Mechelen', 'Li�ge', 'Leuven', 'Oudenaarde', 'Marche-en-Famenne',
       'Antwerp', 'Sint-Niklaas', 'Arlon', 'Neufch�teau', 'Waremme',
       'Verviers', 'Huy', 'Halle-Vilvoorde', 'Bastogne', 'Aalst',
       'Oostend', 'Soignies', 'Brugge', 'Tielt', 'Charleroi', 'Thuin',
       'Virton', 'Kortrijk', 'Mons', 'Turnhout', 'Tournai', 'Namur',
       'Roeselare', 'Mouscron', 'Maaseik', 'Ath', 'Tongeren', 'Eeklo',
       'Veurne', 'Hasselt', 'Philippeville', 'Ieper', 'Diksmuide'],
      dtype=object)